In [7]:
import csv
import pandas as pd
import geopandas as gpd
import pydeck as pdk

In [8]:
# Upload data of each countries' population.
df1 = pd.read_csv("projected-population-by-country.csv")
df1.head(10)

,Entity,Code,Year,Population
0,Afghanistan,AFG,1800,3280000
1,Afghanistan,AFG,1801,3280000
2,Afghanistan,AFG,1802,3280000
3,Afghanistan,AFG,1803,3280000
4,Afghanistan,AFG,1804,3280000
5,Afghanistan,AFG,1805,3280000
6,Afghanistan,AFG,1806,3280000
7,Afghanistan,AFG,1807,3280000
8,Afghanistan,AFG,1808,3280000
9,Afghanistan,AFG,1809,3280000


In [9]:
# Upload data of each countries' latitude and longtitude. 
df2 = "countries.csv"
df2 = pd.read_csv(df2)
df2.head(10)

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
5,AL,41.153332,20.168331,Albania
6,AM,40.069099,45.038189,Armenia
7,AN,12.226079,-69.060087,Netherlands Antilles
8,AO,-11.202692,17.873887,Angola
9,AQ,-75.250973,-0.071389,Antarctica


In [10]:
# Merge two datas.
merged = df2.merge(df1, left_on = 'name', right_on = 'Entity', how = 'left')

# Get data before 2021
merged = merged.loc[merged["Year"] < '2021', :] 

# change object -> value 
merged['Year'] = merged['Year'].astype(int)
merged['Population'] = merged['Population'].astype(int)

# NAN -> string
merged.fillna('No data', inplace = True)

# Nomalize population
merged['normal'] = merged['Population'] / merged['Population'].max()

# Delete column "country, entity, code"
merged.drop(["country", 'Entity', 'Code'], axis='columns', inplace=True)

merged

,latitude,longitude,name,Year,Population,normal
0,42.546245,1.601554,Andorra,1800,2654,0.000002
1,42.546245,1.601554,Andorra,1801,2654,0.000002
2,42.546245,1.601554,Andorra,1802,2654,0.000002
3,42.546245,1.601554,Andorra,1803,2654,0.000002
4,42.546245,1.601554,Andorra,1804,2654,0.000002
...,...,...,...,...,...,...
60010,-19.015438,29.154857,Zimbabwe,2016,14030000,0.009748
60011,-19.015438,29.154857,Zimbabwe,2017,14237000,0.009891
60012,-19.015438,29.154857,Zimbabwe,2018,14439000,0.010032
60013,-19.015438,29.154857,Zimbabwe,2019,14645000,0.010175


In [11]:
initial_data = merged[merged['Year'] == 1800].to_dict(orient= 'records')
initial_data

[{'latitude': 42.546245,
  'longitude': 1.6015540000000001,
  'name': 'Andorra',
  'Year': 1800,
  'Population': 2654,
  'normal': 1.8439211741067335e-06},
 {'latitude': 23.424076,
  'longitude': 53.847818000000004,
  'name': 'United Arab Emirates',
  'Year': 1800,
  'Population': 40153,
  'normal': 2.7897123927621577e-05},
 {'latitude': 33.93911,
  'longitude': 67.709953,
  'name': 'Afghanistan',
  'Year': 1800,
  'Population': 3280000,
  'normal': 0.0022788475701093014},
 {'latitude': 17.060816,
  'longitude': -61.796428000000006,
  'name': 'Antigua and Barbuda',
  'Year': 1800,
  'Population': 37000,
  'normal': 2.5706512223793946e-05},
 {'latitude': 41.153332,
  'longitude': 20.168331,
  'name': 'Albania',
  'Year': 1800,
  'Population': 410445,
  'normal': 0.00028516511918094884},
 {'latitude': 40.069099,
  'longitude': 45.038189,
  'name': 'Armenia',
  'Year': 1800,
  'Population': 413326,
  'normal': 0.00028716675328140155},
 {'latitude': -11.202691999999999,
  'longitude': 17.8

In [12]:
# Set layers 
layer = pdk.Layer(
    "HeatmapLayer",
    initial_data,
    opacity=0.8,
    get_position=["longitude", "latitude"],
    aggregation="ADD",
    get_weight="normal",
    auto_highlight=True,
    pickable=True,
    get_radius=200)

center = [1.6015540000000001, 42.546245] 
view_state = pdk.ViewState( longitude=center[0], latitude=center[1], zoom=1) # Render 
r = pdk.Deck(
    layers=[layer], 
    initial_view_state = view_state ,
    views=[pdk.View(type='MapView', controller=None)],
    tooltip={
        'html': '<b>Nation:</b> {name}',
        'style': {
            'color': 'white'
        }
    }
)


# Make widgets 
import ipywidgets as widgets
from IPython.display import display
slider = widgets.IntSlider(1800, min=1800, max=2020, step=2)

# Define functions for slider 
def on_change(v):
    results = merged[merged['Year'] == slider.value].to_dict(orient='records')
    layer.data = results
    r.update()
        
# Connect deck with slider 
display(slider)
slider.observe(on_change)
r.show()

/opt/anaconda3/lib/python3.7/site-packages/pydeck/bindings/deck.py:88: UserWarning: Mapbox API key is not set. This may impact available features of pydeck.
  UserWarning,


IntSlider(value=1800, max=2020, min=1800, step=2)

DeckGLWidget(json_input='{"initialViewState": {"latitude": 42.546245, "longitude": 1.6015540000000001, "zoom":…